In [1]:
import numpy as np
import pandas as pd
import matplotlib.dates as mdates
import datetime as dt

In [2]:
def tau_x_y(wind_speed,component):
    temp_list = np.zeros(wind_speed.shape)
    for i in range(len(wind_speed)):
        if wind_speed[i]<11:
            temp = (1.3*1.2e-3)*wind_speed[i]*component[i] # EW surface wind stress for low wind speed
            temp_list[i]=temp
        else:
            temp = (1.3*1.2e-3)*(0.49+0.065*wind_speed[i])*wind_speed[i]*component[i] # EW surface wind stress for high wind speed
            temp_list[i]=temp     
    return temp_list

In [3]:
df = pd.read_csv("met_data_svalbard_lufthavn.csv",delimiter=";",skipfooter=1,na_values='-',engine="python")
df = df.drop(columns=["Station"])
df['Time'] = pd.to_datetime(df['Time'], format='%d.%m.%Y %H:%M')

df_precip = pd.read_csv("met_data_svalbard_lufthavn_precip.csv",delimiter=";",skipfooter=1,na_values='-',engine="python")
df_precip['Time'] = pd.to_datetime(df_precip['Time'], format='%d.%m.%Y %H:%M')
df_precip = df_precip.drop(columns=["Name","Station"])
df_precip['Precipitation'] = df_precip['Precipitation']/1000/24/3600

df = pd.merge(df, df_precip, on="Time")

df = df.interpolate()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4764 entries, 0 to 4763
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Name                   4764 non-null   object        
 1   Time                   4764 non-null   datetime64[ns]
 2   Air temperature        4764 non-null   float64       
 3   Mean wind speed        4764 non-null   float64       
 4   Wind direction         4764 non-null   float64       
 5   Cloud cover            4764 non-null   float64       
 6   Relative air humidity  4764 non-null   int64         
 7   Precipitation          4764 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 335.0+ KB


,Name,Time,Air temperature,Mean wind speed,Wind direction,Cloud cover,Relative air humidity,Precipitation
0,Svalbard Lufthavn,2020-10-01 01:00:00,2.0,0.4,262.0,7.000000,93,3.472222e-09
1,Svalbard Lufthavn,2020-10-01 02:00:00,2.2,1.3,300.0,6.666667,94,0.000000e+00
2,Svalbard Lufthavn,2020-10-01 03:00:00,1.6,1.2,177.0,6.333333,95,2.314815e-09
3,Svalbard Lufthavn,2020-10-01 04:00:00,1.7,0.0,0.0,6.000000,96,0.000000e+00
4,Svalbard Lufthavn,2020-10-01 05:00:00,1.7,2.2,229.0,6.333333,90,0.000000e+00


## Wind component conversion from met-coordinates (0 deg from N) to wind stress
## Calculate wind stress based on Large & pond (1981)


In [4]:
df["u10"] = -df["Mean wind speed"]*np.sin(df["Wind direction"]/360*2*np.pi)
df['taux'] = tau_x_y(df['Mean wind speed'],df['u10'])
df["v10"] = -df["Mean wind speed"]*np.cos(df["Wind direction"]/360*2*np.pi)
df['tauy'] = tau_x_y(df['Mean wind speed'],df['v10'])

## Calculate wind stress based on Large & pond (1981)

In [5]:
rhoa = 1.3           # Air density in kg/m3
C    = 2.0e-3        # Heat transfer coefficient (sensible + latent heat flux including evaporation effects)
cp   = 1004          # Specific heat at constant pressure for dry air in J/(deg kg)
Ts   = -1.865         # Ocean surface temperature, assumed constant. -1.865 = freezing point for surface seawater with salinity Sw = 34.
es   = 0.98          # Sea surface emissivity
sig  = 5.67e-8       # Stefan-Boltzmann constant
alpha= 0.1           # Albedo of open water
S0   = 1353          # The sun constant in W/m2
psi  = 77.75         # Latitude of the polynya in Degrees
r    = 7.5           # Constant
b    = 237.3         # Constant

length = df["Time"].to_numpy().shape[0]

In [6]:
FT = (rhoa*C*cp)*df["Mean wind speed"]*(df["Air temperature"]-Ts) # Turbulent flux (sensible + latent). Ta-Ts, differensial=>can use degree Celsius
FL = -es*(sig*(Ts*np.ones(length)+273.15)**4)                       # Balckbody radiation form the ocean. Ts+273.15=degree Kelvin
ea = 0.7829*(1+0.2232*(df["Cloud cover"]/8)**(2.75))               # The effective air emissivity
FB = (ea*(sig*(df["Air temperature"]+273.15)**4))                  # Balckbody radiation form the atms. Simonsen & Haugan (1996), in deg. Kelvin
k  = 1-0.6*(df["Cloud cover"]/8)**3                                # Cloud correction term
vp = (df["Relative air humidity"]/100)*6.11*10**(r*df['Air temperature']/(b+df['Air temperature']))       # Vapor pressure

In [7]:
day_of_year = df["Time"].dt.day_of_year
year = df['Time'].dt.year
month = df['Time'].dt.month
day = df['Time'].dt.day
hour = df["Time"].dt.hour
#print(year,month,day,hour,day_of_year)

i  = 23.44*np.cos((360/365)*(172-day_of_year)*2*np.pi/360)
hour_s = hour+2                               #Sun hour = 2 hours behind local time
sun_hour_angle  = (12-hour_s)*15                                  #Sun hour angle

coszp = np.sin(psi*np.pi/180)*np.sin(i*np.pi/180)+np.cos(psi*np.pi/180)*np.cos(i*np.pi/180)*np.cos(sun_hour_angle)
coszp[coszp<0]=0
Q0 = ((S0*coszp**2)/(1.085*coszp+(2.7+coszp)*vp*1e-3+0.1))
FS = (1-alpha)*k*Q0
Fnet = FT+FL-FB-FS

In [8]:
Bowen_ratio = 1.

In [9]:
output = pd.DataFrame()
output['time']=df['Time']
output['sw'] = FS
output['lw'] = FL + FB
output['qsens'] = FT
output['qlat'] = FT/Bowen_ratio
output['tx'] = df['taux']
output['ty'] = df['tauy']
output['precip'] = df['Precipitation']

start_date = dt.date(2021, 1, 14)
end_date = dt.date(2021, 4, 1)

output = output[mdates.date2num(output['time'])>=mdates.date2num(start_date)]
output = output[mdates.date2num(output['time'])<=mdates.date2num(end_date)]

output['time'] = mdates.date2num(output['time'])-mdates.date2num(start_date)
output = output.iloc[::3]
print(output.iloc[110:120])

output_xr = output.to_xarray()
output_xr.to_netcdf(path="../input_data/Svalbard_Lufthavn.nc", mode='w')
print(output_xr)

        time   sw          lw       qsens        qlat        tx        ty  \
2828  13.750  0.0  -99.361834 -205.241395 -205.241395 -0.052041  0.046858   
2831  13.875  0.0  -99.361834 -235.874439 -235.874439 -0.074828  0.054366   
2834  14.000  0.0 -102.143556 -244.069790 -244.069790 -0.064472  0.056044   
2837  14.125  0.0 -106.413335 -307.750498 -307.750498 -0.087909  0.066244   
2840  14.250  0.0 -111.208461 -379.566517 -379.566517 -0.118008  0.065413   
2843  14.375  0.0 -109.719618 -438.543284 -438.543284 -0.183496  0.143363   
2846  14.500  0.0 -108.989352 -478.817841 -478.817841 -0.241006  0.175101   
2849  14.625  0.0 -108.316701 -499.024947 -499.024947 -0.286547  0.179055   
2852  14.750  0.0 -113.376258 -455.952042 -455.952042 -0.145856  0.091141   
2855  14.875  0.0 -116.914619 -543.205967 -543.205967 -0.212191  0.171829   

      precip  
2828     0.0  
2831     0.0  
2834     0.0  
2837     0.0  
2840     0.0  
2843     0.0  
2846     0.0  
2849     0.0  
2852     0.0  
28